# SVMLIGHT loading

In [1]:
using SparseArrays

function load_svmlight(filename)
    b = Float64[]
    I = Int[]
    J = Int[]
    V = Float64[]

    line_idx = 0

    file = open(filename)
    
    for line in eachline(file)
        line_idx += 1
    
        line_split = split(line)
        b_i = parse(Float64,line_split[1])
        push!(b, b_i)
        
        line_split = line_split[2:end]
    
        for tuple in line_split
            tuple_split = split(tuple, ':')
            push!(I, line_idx)
            push!(J, parse(Int, tuple_split[1]))
            push!(V, parse(Float64, tuple_split[2]))
        end
    end

    A = sparse(I,J,V)
    return A, b
end

load_svmlight (generic function with 1 method)

# Oracle

In [2]:
module oracles

using SparseArrays
import LinearAlgebra.norm_sqr
abstract type BaseOracle end
    

mutable struct L2Oracle <: BaseOracle
    func::Function
    grad::Function
    
    
    full_func::Function
    full_grad::Function
    
    A::SparseArrays.SparseMatrixCSC
    b::Array{Float64,1}
    reg::Float64
    
    n::Int
    d::Int
    
    μ::Float64
    L::Float64
    
    Atb::Array{Float64,1}
    AtA::SparseArrays.SparseMatrixCSC
    
    function L2Oracle(A_, b_, reg_)
        this = new()
        
        this.A = A_
        this.b = b_
        this.reg = reg_
        (this.n, this.d) = size(this.A)
        
        
        this.L = maximum([norm_sqr(this.A[i,:]) for i in 1:this.n])
        this.L += reg_
        this.μ = reg_
        
        
        this.func = function(i, x)
            a_i = this.A[i,:]
            b_i = this.b[i]
            return (a_i'x - b_i) ^ 2 / 2 + this.reg / 2.0 * x'x
        end
        
        this.grad = function(i, x)
            a_i = this.A[i,:]
            b_i = this.b[i]
            return (a_i'x - b_i) * a_i + this.reg * x
        end
        
        
        this.AtA = this.A'this.A
        this.Atb = this.A'this.b
        
        this.full_func = function(x)
            Ax_b = this.A * x - this.b
            return Ax_b'Ax_b / 2.0 / this.n + x'x * this.reg / 2.0
        end
        
        this.full_grad = function(x)
            return (this.AtA * x - this.Atb) / this.n + this.reg * x
        end
        
        println("L2 oracle created")
        println("\tn = $(this.n); d = $(this.d); L = $(this.L); μ = $(this.μ)")
        println("\tκ = $(this.L / this.μ)")
        
        return this
    end
end


mutable struct LogRegOracle <: BaseOracle
    func::Function
    grad::Function
    
    
    full_func::Function
    full_grad::Function
    
    A::SparseArrays.SparseMatrixCSC
    b::Array{Float64,1}
    reg::Float64
    
    n::Int
    d::Int
    
    μ::Float64
    L::Float64
    
    Atb::Array{Float64,1}
    AtA::SparseArrays.SparseMatrixCSC
    
    function LogRegOracle(A_, b_, reg_)
        this = new()
        
        this.A = A_
        this.b = b_
        this.reg = reg_
        (this.n, this.d) = size(this.A)
        
        
        this.L = maximum([norm_sqr(this.A[i,:]) for i in 1:this.n]) / 4.0
        this.L += reg_
        this.μ = reg_
        
        
        this.func = function(i, x)
            a_i = this.A[i,:]
            b_i = this.b[i]
            return log(1.0 + exp(-b_i*a_i'x)) + this.reg / 2.0 * x'x
        end
        
        this.grad = function(i, x)
            a_i = this.A[i,:]
            b_i = this.b[i]
            return (-b_i * a_i) / (1.0 + exp(b_i * a_i'x)) + this.reg * x
        end
        
        this.full_func = function(x)
            return sum([this.func(i,x) for i in 1:this.n]) / this.n
        end
        
        this.full_grad = function(x)
            return sum([this.grad(i,x) for i in 1:this.n]) / this.n
        end
        
        println("Logistic regression oracle created")
        println("\tn = $(this.n); d = $(this.d); L = $(this.L); μ = $(this.μ)")
        println("\tκ = $(this.L / this.μ)")
        
        return this
    end
end


end

Main.oracles

# ProgressBar

In [3]:
module progress

mutable struct progress_bar
    mval::Float64
    cnt::Int64
    num::Int64
    title::String
    
    update::Function
    
    function progress_bar(mval_, title_, num_=20)
        this = new()
        
        this.mval = mval_
        this.cnt = 0
        this.num = num_
        this.title = title_
        
        println(this.title)
        print("Progress: [")
        for i in 1:this.num
            print("-")
        end
        print("]\r")
        flush(stdout)
        this.update = function(val)
            idx = Int(round(val / this.mval * Float64(this.num)))
            if this.cnt >= this.num
                return
            end
            
            if idx > this.cnt
                this.cnt = min(idx, this.num)
                print("Progress: [")
                for i in 1:this.num
                    if i <= this.cnt
                        print("=")
                    else
                        print("-")
                    end
                end
                if this.cnt < this.num
                    print("]\r")
                else
                    print("]\n")
                end
                flush(stdout)
                
            end
        end
        
        return this
    end
end
end

Main.progress

# Methods

# SAGA

In [4]:
import LinearAlgebra.norm2

function SAGA(oracle::oracles.BaseOracle, num_passes, x_0)
    history = Dict("k" => 0.0, "r" => Array[])
    pbar = progress.progress_bar(num_passes, "SAGA")
    
    x_k = copy(x_0)
    
    D = zeros(oracle.d, oracle.n)
    full_g = zeros(oracle.d)
    
    for i in 1:oracle.n
        D[:, i] = oracle.grad(i, x_0)
        full_g += D[:, i] / oracle.n
    end
    
    
    history["k"] += 1.0
    
    while history["k"] < num_passes
        i = rand(1:oracle.n)
        
        grad_i = oracle.grad(i, x_k)
        
        g_k = grad_i - D[:, i] + full_g
        full_g += (grad_i - D[:,i]) / oracle.n
        D[:,i] = grad_i
        
        x_k = x_k - g_k / (6.0 * oracle.L)
        
        history["k"] += 1.0 / oracle.n
        if history["k"] > length(history["r"]) 
            push!(history["r"], x_k)
        end
        
        pbar.update(history["k"])
    end
    return history
end

SAGA (generic function with 1 method)

# SVRG

In [5]:
import LinearAlgebra.norm2

function SVRG(oracle::oracles.BaseOracle, num_passes, x_0; c_len=oracle.n)
    history = Dict("k" => 0.0, "r" => Array[])
    
    pbar = progress.progress_bar(num_passes, "SVRG")
    
    x_k = copy(x_0)
    w_k = copy(x_0)
    
    
    while history["k"] < num_passes
        full_g = oracle.full_grad(w_k)
        history["k"] += 1.0
        
        m = rand(1:2*c_len)
        
        for j in 1:m
            i = rand(1:oracle.n)
            g_k = oracle.grad(i, x_k) - oracle.grad(i, w_k) + full_g
            history["k"] += 2.0 / oracle.n
        
            x_k -= (0.1 / oracle.L) * g_k
        
            if history["k"] > length(history["r"]) 
                push!(history["r"], x_k)
            end
            pbar.update(history["k"])
        end
        w_k = copy(x_k)
    end
    return history
end

SVRG (generic function with 1 method)

# Stupid Method

In [6]:
import LinearAlgebra.norm2

function SM(oracle::oracles.BaseOracle, num_passes, x_0; p = 1.0 / oracle.n)
    history = Dict("k" => 0.0, "r" => Array[])
    pbar = progress.progress_bar(num_passes, "SM")
    
    x_k = copy(x_0)
    w_k = copy(x_0)
    
    full_g = oracle.full_grad(w_k)
    history["k"] += 1.0
    
    while history["k"] < num_passes
        if rand() < 1.0 / oracle.n
            w_k = copy(x_k)
            full_g = oracle.full_grad(w_k)
            history["k"] += 1.0
        end
        
        i = rand(1:oracle.n)
        g_k = oracle.grad(i, x_k) - oracle.grad(i, w_k) + full_g
        x_k = x_k - g_k / (6.0 * oracle.L)
        
        history["k"] += 2.0 / oracle.n
        if history["k"] > length(history["r"]) 
            push!(history["r"], x_k)
        end
        
        pbar.update(history["k"])
    end
    return history
end

SM (generic function with 1 method)

# Katyusha

In [7]:
import LinearAlgebra.norm2

function Katyusha(oracle::oracles.BaseOracle, num_passes, x_0)
    history = Dict("k" => 0.0, "r" => Array[])
    
    pbar = progress.progress_bar(num_passes, "Katyusha")
    
    m = 2 * oracle.n
    σ = oracle.μ / oracle.L
    θ_1 = min(0.5, sqrt(2.0 * σ * oracle.n / 3.0))
    θ_2 = 1/2
    η = 1.0 / (3.0 * θ_1)
    
    y_k = copy(x_0)
    z_k = copy(x_0)
    w_k = copy(x_0)
    
    
    while history["k"] < num_passes
        full_g = oracle.full_grad(w_k)
        history["k"] += 1.0
        
        new_w = zeros(oracle.d)
        norm_coef = 0.0
        for j in 1:m
            x_k = θ_1 * z_k + θ_2 * w_k + (1.0 - θ_1 - θ_2) * y_k
        
            i = rand(1:oracle.n)
            g_k = oracle.grad(i, x_k) - oracle.grad(i, w_k) + full_g
            history["k"] += 2.0 / oracle.n
        
        
            y_k = x_k - g_k / (3.0 * oracle.L)
            z_k = (z_k + η * σ * x_k - η * g_k / oracle.L) / (1.0 + η * σ)
            
            
            #z_old = copy(z_k)
            #z_k = (z_k + η * σ * x_k - η * g_k / oracle.L) / (1.0 + η * σ)
            #y_k = x_k + θ_1 * (z_k - z_old)
            
            
            new_w += y_k * (1.0 + η * σ)^j
            norm_coef += (1.0 + η * σ)^j
        
            if history["k"] > length(history["r"]) 
                push!(history["r"], y_k)
            end
            pbar.update(history["k"])
        end
        w_k = new_w / norm_coef
    end
    return history
end

Katyusha (generic function with 1 method)

# Accelerated Stupid Method

In [8]:
import LinearAlgebra.norm2

function ASM(oracle::oracles.BaseOracle, num_passes, x_0)
    history = Dict("k" => 0.0, "r" => Array[])
    pbar = progress.progress_bar(num_passes, "ASM")
    
    σ = oracle.μ / oracle.L
    θ_1 = min(0.5, sqrt(2.0 * σ * oracle.n / 3.0))
    θ_2 = 1/2
    η = 1.0 / (3.0 * θ_1)
    
    y_k = copy(x_0)
    z_k = copy(x_0)
    w_k = copy(x_0)
    
    full_g = oracle.full_grad(w_k)
    history["k"] += 1.0
    
    while history["k"] < num_passes
        x_k = θ_1 * z_k + θ_2 * w_k + (1.0 - θ_1 - θ_2) * y_k
        
        i = rand(1:oracle.n)
        g_k = oracle.grad(i, x_k) - oracle.grad(i, w_k) + full_g
        history["k"] += 2.0 / oracle.n
        
        
        if rand() < 1.0 / oracle.n
            w_k = copy(y_k)
            full_g = oracle.full_grad(w_k)
            history["k"] += 1.0
        end
    
        z_old = copy(z_k)
        z_k = (z_k + η * σ * x_k - η * g_k / oracle.L) / (1.0 + η * σ)
        y_k = x_k + θ_1 * (z_k - z_old)
        
        ### y_k!!!!!
        if history["k"] > length(history["r"]) 
            push!(history["r"], y_k)
        end
        pbar.update(history["k"])
    end
    return history
end

ASM (generic function with 1 method)

# Solver

In [9]:
using LinearAlgebra
using JLD2
function Solve(dataset, reg_power, num_pass; dataset_path="datasets/", dump_path="dump/")
    println("-----------------")
    println("Solve start:")
    println("Dataset = ", dataset)
    println("L2 regularizer = 10^-", reg_power)
    
    
    A, b = load_svmlight(string(dataset_path, dataset))
    
    oracle = oracles.LogRegOracle(A, b, 0.1^reg_power);
    
    x_star = zeros(oracle.d)
    jldopen(string(dump_path, dataset,"_",reg_power,"_solution"), "a+") do dump
        x_star = get(dump, "x_star", zeros(oracle.d))
    end
    println(LinearAlgebra.norm_sqr(oracle.full_grad(x_star)))
    
    @time h = ASM(oracle, num_pass, x_star);
    x_star = h["r"][length(h["r"])]
    
    println(LinearAlgebra.norm_sqr(oracle.full_grad(x_star)))
    jldopen(string(dump_path, dataset,"_",reg_power,"_solution"), "w") do dump
        dump["x_star"] = x_star
    end
end

Solve (generic function with 1 method)

In [203]:
for i in 1:50
    println("ITERATION ", i)
    Solve("mushrooms", 3, 100);
end

ITERATION 1
-----------------
Solve start:
Dataset = mushrooms
L2 regularizer = 10^-3
Logistic regression oracle created
	n = 8124; d = 112; L = 5.251; μ = 0.0010000000000000002
	κ = 5250.999999999999
1.299891160034541e-30
ASM2


InterruptException: InterruptException:

# Experiment

In [10]:
using LinearAlgebra
using JLD2
function experiment(dataset, reg_power, num_pass; dataset_path="datasets/", dump_path="dump/")
    println("-----------------")
    println("Experiment start:")
    println("Dataset = ", dataset)
    println("L2 regularizer = 10^", -reg_power)
    
    
    A, b = load_svmlight(string(dataset_path, dataset))
    #avg_norm = sum([norm2(A[i,:]) for i in 1:size(A)[1]]) / Float64(size(A)[1])
    #A /= avg_norm
    
    oracle = oracles.LogRegOracle(A, b, 0.1^reg_power);
    
    #x_star = zeros(oracle.d)
    #jldopen(string(dump_path, dataset,"_",reg_power,"_solution"), "a+") do dump
    #    x_star = dump["x_star"] 
    #end
    #println(LinearAlgebra.norm_sqr(oracle.full_grad(x_star)))
    
    #@time x_star = Solve(oracle, x_star, 100)
    
    #println(LinearAlgebra.norm_sqr(oracle.full_grad(x_star)))
    #jldopen(string(dump_path, dataset,"_",reg_power,"_solution"), "w") do dump
    #    dump["x_star"] = x_star
    #end
    
    #@time hSAGA = SAGA(oracle, num_pass, zeros(oracle.d));
    #jldopen(string(dump_path, dataset,"_",reg_power), "a+") do dump
    #    dump["SAGA"] = hSAGA
    #end
    
    @time hASM = ASM(oracle, num_pass, zeros(oracle.d));
    jldopen(string(dump_path, dataset,"_",reg_power), "a+") do dump
        dump["ASM"] = hASM
    end
    
    @time hSM = SM(oracle, num_pass, zeros(oracle.d));
    jldopen(string(dump_path, dataset,"_",reg_power), "a+") do dump
        dump["SM"] = hSM
    end
    
    @time hSVRG = SVRG(oracle, num_pass, zeros(oracle.d));
    jldopen(string(dump_path, dataset,"_",reg_power), "a+") do dump
        dump["SVRG"] = hSVRG
    end
    
    @time hKatyusha = Katyusha(oracle, num_pass, zeros(oracle.d));
    jldopen(string(dump_path, dataset,"_",reg_power), "a+") do dump
        dump["Katyusha"] = hKatyusha
    end
end

experiment (generic function with 1 method)

In [11]:
experiment("w8a", 1, 100)
experiment("w8a", 2, 100)
experiment("a9a", 2, 100)
experiment("a9a", 3, 100)

-----------------
Experiment start:
Dataset = w8a
L2 regularizer = 10^-1
Logistic regression oracle created
	n = 49749; d = 300; L = 28.6; μ = 0.1
	κ = 286.0
ASM
Progress: [====================]
151.765618 seconds (247.43 M allocations: 252.824 GiB, 14.28% gc time)
SM
Progress: [====================]
122.055188 seconds (175.95 M allocations: 120.923 GiB, 11.80% gc time)
SVRG
Progress: [====================]
125.292022 seconds (182.33 M allocations: 123.740 GiB, 12.46% gc time)
Katyusha
Progress: [====================]
165.515387 seconds (270.87 M allocations: 296.238 GiB, 11.83% gc time)
-----------------
Experiment start:
Dataset = w8a
L2 regularizer = 10^-2
Logistic regression oracle created
	n = 49749; d = 300; L = 28.51; μ = 0.010000000000000002
	κ = 2850.9999999999995
ASM
Progress: [====================]
157.651456 seconds (252.15 M allocations: 267.061 GiB, 14.57% gc time)
SM
Progress: [====================]
128.137507 seconds (176.40 M allocations: 122.621 GiB, 16.72% gc time)
S

Dict{String,Any} with 2 entries:
  "k" => 100.0
  "r" => Array[[-0.00903979, -0.00584377, -0.00403815, -0.00232903, -0.00342702…

# Experiment 2

In [208]:
using LinearAlgebra
using JLD2
function experiment2(dataset, reg_power, num_pass; dataset_path="datasets/", dump_path="dump/")
    println("-----------------")
    println("Experiment start:")
    println("Dataset = ", dataset)
    println("L2 regularizer = 10^", -reg_power)
    
    
    A, b = load_svmlight(string(dataset_path, dataset))
    oracle = oracles.LogRegOracle(A, b, 0.1^reg_power);
    
    #h = SVRG(oracle, num_pass, zeros(oracle.d))
    #jldopen(string(dump_path, dataset,"_",reg_power, "_p"), "a+") do dump
    #    dump["SVRG"] = h
    #end
    
    κ = oracle.L / oracle.μ
    n = oracle.n
    
    m = [n, sqrt(sqrt(κ * n ^ 3 )), sqrt(κ * n), sqrt(sqrt(n * κ ^ 3 )), κ]
    println(m)
    
    for i in 1:length(m)
        h = SVRG(oracle, num_pass, zeros(oracle.d), c_len = m[i])
        jldopen(string(dump_path, dataset,"_",reg_power, "_p"), "a+") do dump
            dump[string("S_", i)] = h
        end
    end
    
    for i in 1:length(m)
        h = SM(oracle, num_pass, zeros(oracle.d), p = 1.0 / m[i])
        jldopen(string(dump_path, dataset,"_",reg_power, "_p"), "a+") do dump
            dump[string("L_", i)] = h
        end
    end
    #for k in [0.2,0.5,1.0,2.0,5.0]
    #    h = SM(oracle, num_pass, zeros(oracle.d), p = k / oracle.n)
    #    jldopen(string(dump_path, dataset,"_",reg_power, "_p"), "a+") do dump
    #        dump[string("SM_", k, "_n")] = h
    #    end
    #end
    
    #for k in [0.2,0.5,1.0,2.0,5.0]
    #    h = SM(oracle, num_pass, zeros(oracle.d), p = k * oracle.μ / oracle.L)
    #    jldopen(string(dump_path, dataset,"_",reg_power, "_p"), "a+") do dump
    #        dump[string("SM_", k, "_k")] = h
    #    end
    #end
    
end

experiment2 (generic function with 1 method)

In [212]:
experiment2("phishing", 4, 100)

-----------------
Experiment start:
Dataset = phishing
L2 regularizer = 10^-4
Logistic regression oracle created
	n = 11055; d = 68; L = 0.2500885367499999; μ = 0.00010000000000000002
	κ = 2500.8853674999987
[11055.0, 7624.17, 5258.07, 3626.27, 2500.89]
SVRG
Progress: [====================]
SVRG
Progress: [====================]
SVRG
Progress: [====================]
SVRG
Progress: [====================]
SVRG
Progress: [====================]
SM
Progress: [====================]
SM
Progress: [====================]
SM
Progress: [====================]
SM
Progress: [====================]
SM
Progress: [====================]
